In [1]:
import numpy as np
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns;
sns.set()

There are 2 kinds of Confidence Intervals ( and we use 2 different standard error to measure them )

1. CI for Predicted Mean (the mean of predicted values) :

This is usually known as Confidence Interval, and this measures tge Variance (Std Error ^ 2) of the expected value of the prediction

   =>  Var(Y_hat|X=Xh) = Var(Beta0_hat + Beta1_hat * Xh) = Var(Prediction Mean)

2. CI for Predicted Observations (or Predicted Value)   :

This is also called Prediction/Forecast Interval, and this directly measures the Variance (Std Error ^ 2) of thhe prediction for NEW OBSERVATION (Y_hat, X_hat).

   =>  Var(Pred|X_Xh) = Var(ei|X=Xh) = Var{ (Y_hat|X=Xh) - (Y|X=Xh) } 
   
   = Var(Y_hat|X=Xh)                 + Var(Y|X=Xh)                  - 2Cov(Y_hat|X=Xh, Y|X=Xh)
   
   = Var(Beta0_hat + Beta1_hat * Xh) + Var(Beta0 + Beta1 * Xh + εh) - 0  
   
   <- 0 because the True value of Y for the new observation is completely NEW to the training set, so they do not have any correlation
   
   = Var(Prediction Mean) + Var(   εh   ) 
   
   = Var(Prediction Mean) + Var(Residual)



## Generate a Tool Data 

In [6]:
A  = np.linspace(0, 100, 1000)

WN = np.random.normal(loc=0, scale=5, size=1000) # White Noise

B = 50 + 0.1 * A + WN # True Model

## The simplest way to get CI for Mean/Obs, use "summary_frame()" 

In [11]:
X_train = sm.add_constant(A)
Y_train = B

model = OLS(endog=Y_train, exog=X_train, missing='None', hasconst=True) # define the OLS object
result = model.fit()                                                    # fit the model

preds_ = result.get_prediction(X_train)                                 # then we get a linear_model.PredictionResult object

preds_.summary_frame(alpha=0.05).tail()                                 # Now we can see the CI for Mean & CI for Obs

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
995,59.886954,0.310550,59.277547,60.496360,50.166863,69.607045
996,59.897028,0.311018,59.286703,60.507353,50.176880,69.617177
997,59.907103,0.311487,59.295859,60.518347,50.186896,69.627309
998,59.917177,0.311955,59.305014,60.529340,50.196913,69.637441
999,59.927251,0.312424,59.314168,60.540335,50.206929,69.647574


## Alternative Method I

In [12]:
# 95% Confidence Intervals for Predicted Mean
mean_ci_upper_1 = preds_.predicted_mean + 1.96 * preds_.se_mean
mean_ci_lower_1 = preds_.predicted_mean - 1.96 * preds_.se_mean

# 95% Confidence Intervals for Prediction Obs
obs_ci_upper_1  = preds_.predicted_mean + 1.96 * preds_.se_obs
obs_ci_lower_1  = preds_.predicted_mean - 1.96 * preds_.se_obs

## Alternative Method II 

In [13]:
# 95% Confidence Intervals for Predicted Mean
mean_ci_upper_2 = preds_.predicted_mean + 1.96 * preds_.var_pred_mean
mean_ci_lower_2 = preds_.predicted_mean - 1.96 * preds_.var_pred_mean

# 95% Confidence Intervals for Prediction Obs
obs_ci_upper_2  = preds_.predicted_mean + 1.96 * np.sqrt(preds_.var_pred_mean + preds_.var_resid - 2*0 )
obs_ci_lower_2  = preds_.predicted_mean - 1.96 * np.sqrt(preds_.var_pred_mean + preds_.var_resid - 2*0 )

## Alternative Method III 

In [14]:
# 95% Confidence Intervals for Predicted Mean
mean_ci_3 = preds_.conf_int(obs=False, alpha=0.05)

# 95% Confidence Intervals for Prediction Obs
obs_ci_3  = preds_.conf_int(obs=True , alpha=0.05)
